In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2, suppress=True)
%precision 2


F = 4500 # N
v = 3.05 # m/s
D = 500 # mm
L = 4 # year
t1 = 12 # s
t2 = 60 # s
T1 = 1 # N.mm
T2 = 0.7 # N.mm
def disp(*args):
    a = []
    for arg in args:
        a.append(arg)
    return np.array(a,dtype=float)

fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fkses1 = interp1d([400,500,600,700,800,900,1000,1200],
            [2.25,2.5,2.75,3,3.25,3.5,3.75,4.25])
fkses2 = interp1d([400,500,600,700,800,900,1000,1200],
             [2.75,3.05,3.36,3.66,3.96,4.28,4.6,5.2])
fes = interp1d([15,20,30,40,50,70,80,100],
              [0.87,0.83,0.77,0.73,0.7,0.66,0.64,0.62])
fet = interp1d([15,20,30,40,50,70,80,100],
               [0.92,0.89,0.81,0.78,0.76,0.73,0.71,0.7])

# fes(850),fes(850)

## Motor Design $(power, rpm)$

In [2]:
# calculate efficiency, e = eta
e_coupling = 1
e_bearing = 0.99
e_helical_gear = 0.96
e_ch = 0.95
e_sys = e_coupling * e_bearing**3 * e_helical_gear * e_ch
disp(e_sys)

P_max = F*v/1000 # calculate max power of the belt conveyor, kW
P_work = P_max*np.sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_motor = P_work/e_sys # lower limit of the motor's power, kW
disp(P_max, P_work, P_motor)

n_belt = v*6e4/np.pi/D # calculate rotational speed of the belt conveyor, rpm
u_ch = 5
u_hg = 5
u_sys = u_ch * u_hg # calculate transmission ratio of the system
n_motor = u_sys * n_belt # calculate theoretical rotational speed of the motor, rpm
disp(n_belt, u_sys, n_motor)
P_motor = 18.5 # power based on choice of motor, kW
n_motor = 2930 # rotational speed based on choice of motor, rpm
u_sys = n_motor / n_belt # new transmission ratio
u_ch = u_sys / u_hg # new transmission ratio of ch drive, given u_hg=const
disp(u_sys, u_ch)

P_ch = P_max
P_sh2 = P_ch / (e_ch*e_bearing) # power of shaft 2, kW
P_sh1 = P_sh2 / (e_helical_gear*e_bearing) # power of shaft 1, kW
P_motor = P_sh1 / (e_coupling) # power of motor, kW
n_sh1 = n_motor # rotational speed of shaft 1, rpm
n_sh2 = n_sh1 / (u_hg) # rotational speed of shaft 2, rpm
T_motor = 9.55e6 * P_motor / n_motor # torque of motor, N.mm
T_sh1 = 9.55e6 * P_sh1 / n_sh1 # torque of shaft 1, N.mm
T_sh2 = 9.55e6 * P_sh2 / n_sh2 # torque of shaft 2, N.mm
disp( P_motor, P_sh1, P_sh2)
disp(  n_motor,n_sh1, n_sh2)
disp(T_motor,T_sh1, T_sh2)

array([0.88])

array([13.72, 10.41, 11.76])

array([ 116.5 ,   25.  , 2912.54])

array([25.15,  5.03])

array([15.35, 15.35, 14.59])

array([2930., 2930.,  586.])

array([ 50047.56,  50047.56, 237825.99])

## Chain Drive Design

In [3]:
# find z1, z2
z_1 = np.ceil(29 - 2*u_ch)
z_2 = np.ceil(u_ch*z_1) + 1 # eq (5.1), p.80, plus 1 to obtain odd number of teeth

# find k, k_z, k_n
n01 = 600 # ~ n_sh2
k_z, k_n = 25/z_1, n01/n_sh2 # eq (5.3), p.81
k_0, k_a, k_dc, k_bt, k_d, k_c = 1, 1, 1, 1, 1.25, 1.3
k = k_0 * k_a * k_dc * k_bt * k_d * k_c # table 5.6, p.82
P_allowance = P_ch * k * k_z * k_n # equals 30.047 <= 42
P_allowance, p, dc, B = 42, 31.75, 9.55, 27.46 # table 5.5, p.81
d_1 = p/np.sin(np.pi/z_1) # examine table 5.8, p<p_max~31.75
d_2 = p/np.sin(np.pi/z_2) # examine table 5.8, p<p_max~31.75
a_min, a_max, a = 30*p, 50*p, 1000 # 952.5<a<1587.5, choose a = 1000
x = 2*a/p + (z_1+z_2)/2 + (z_2-z_1)**2*p/(4*np.pi**2*a) # eq (5.12), p.85
x_c = np.ceil(x)
a = 0.25*p*(x_c-0.5*(z_2+z_1)+np.sqrt((x_c-0.5*(z_2+z_1))**2-2*((z_2-z_1)/np.pi)**2)) - 0.003*a # eq (5.13), p.85
i = z_1*n_sh2/(15*x) # eq (5.14), p.85; i<[i]=25 at table 5.9, p.85
B_min, d_O, d_l, h_max, b_max, Q, q = 15.88, 7.95, 15.88, 24.2, 38, 56.7, 2.6 # table 5.2, p.78
k_f, k_d = 4, 1.2
v_1 = n_sh2*p*z_1/6e4
F_t, F_v, F_O = 1e3*P_ch/v_1, q*v_1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*F_t + F_O + F_v)
s >= 10.3 # table 5.10, p.86
n1 = np.array([1250,1000,900,800,630,500,400,300])
pm = np.array([12.7,15.875,19.5,25.4,31.75,38.1,44.5,50.8])
f = interp1d(n1,pm,kind='cubic')
k_x = 1.15
F_r = k_x*F_t # eq (5.20), p.88
zv = 0.85*v_1**.1
disp(i,v_1,F_t,F_v,F_O,s,F_r, F_O+F_v, F_O+F_v+F_t,zv)

True

array([   5.9 ,    5.89, 2329.53,   90.25,  101.92,   18.98, 2678.96,
        192.17, 2521.7 ,    1.01])

In [4]:
H1, H2, NFO1, NFO2, Lh, alp  = 250, 240, 4e6,4e6, 8*2*300*4, 20
NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
NHE1, NHE2 = 60*n_sh1*Lh*(T1**3*t1 + T2**3*t2)/(t1+t2), 60*n_sh2*Lh*(T1**3*t1 + T2**3*t2)/(t1+t2)
NFE1, NFE2 = 60*n_sh1*Lh*(T1**6*t1 + T2**6*t2)/(t1+t2), 60*n_sh2*Lh*(T1**6*t1 + T2**6*t2)/(t1+t2)
KHL1, KHL2 = (NHO1/NHE1)**(1/6), (NHO2/NHE2)**(1/6)
KFL1, KFL2 = (NFO1/NFE1)**(1/6), (NFO2/NFE2)**(1/6)
KHL1, KHL2, KFL1, KFL2 = 1,1,1,1
sigH1, sigH2 = 570/1.1*KHL1, 550/1.1*KHL2
sigF1, sigF2 = 450*KFL1/1.75, 432*KFL2/1.75
sigH = np.average([sigH1,sigH2])
# disp(NHE1, NHE2, NFE1, NFE2)
# disp(KHL1, KHL2, KFL1, KFL2, sigH1, sigH2, sigF1, sigF2, sigH)
psiba = 0.5
psibd = 0.53*psiba*(u_hg+1)
aw = 43*(u_hg+1)*(T_sh1*1.108/sigH**2/u_hg/psiba)**(1/3)
aw = 125
m, bet = 1.5, 14
z1 = 2*aw*np.cos(bet*np.pi/180) / (m*(u_hg+1))
z1 = 27
z2 = u_hg*z1
bw = psiba*aw
# fig, [ax1,ax2] = plt.subplots(nrows=2)
# ax1.plot(m*(z1+z2)/2/aw)
# ax2.plot(bet)
# disp(np.arccos(min(m*(z1+z2)/2/aw))*180/np.pi)
# disp(z1)
# disp(bw,m*(z1+z2)/aw)
bet = np.arccos(m*(z1+z2)/2/aw)
d1 = dw1 = m*z1/np.cos(bet)
d2 = dw2 = m*z2/np.cos(bet)
da1 = d1+2*(1+0.3)*m
da2 = d2+2*(1-0.3)*m
df1 = d1-(2.5-2*0.3)*m
df2 = d2-(2.5-2*-0.3)*m
db1 = d1*np.cos(alp*np.pi/180)
db2 = d2*np.cos(alp*np.pi/180)
alp_t = alp_tw = np.arctan(np.tan(alp*np.pi/180)/np.cos(bet))
v = np.pi*d1*n_sh1/6e4
zv = 0.85*v**.1
ys = 1.08-0.0695*np.log(m)
sigHH, sigFF1, sigFF2 = sigH*zv, sigF1*1.1*ys, sigF2*1.1*ys
disp(np.round([d1,d2,da1,da2,df1,df2,db1,db2,
                alp_t*180/np.pi,bet*180/np.pi,v,zv,ys,
               sigHH, sigFF1, sigFF2],2))

array([[ 41.67, 208.33,  45.57, 210.43,  38.82, 203.68,  39.15, 195.77,
         20.53,  13.59,   6.39,   1.02,   1.05, 520.93, 297.51, 285.61]])

In [5]:
bet_b = np.arctan(np.cos(alp_t)*np.tan(bet))
zm, zh = 274, np.sqrt(2*np.cos(bet_b)/np.sin(2*alp_tw))
epbet = bw*np.sin(bet)/m/np.pi
epalp = (np.sqrt(da1**2-db1**2)+np.sqrt(da2**2-db2**2)-2*aw*np.sin(alp_tw))/(2*np.pi*m*np.cos(alp_t)/np.cos(bet))
disp(bet_b*180/np.pi,zh,epalp,epbet)
ze = epalp**-0.5
kh, kf, mn = 1.07*1.1*1.108, 1.18*1.29*1.2558, m*np.cos(bet)
sigHc = zm*zh*ze*np.sqrt(2*T_sh1*kh*(u_hg+1)/bw/u_hg/dw1**2)
ye, yb = epalp**-1, 1-bet*180/np.pi/140
zv1, zv2, yf1, yf2 = z1/np.cos(bet)**3, z2/np.cos(bet)**3, 3.54, 3.63
sigF1 = 2*T_sh1*kf*ye*yb*yf1/bw/dw1/mn
sigF2 = sigF1*yf2/yf1
Ft = 2*T_sh1/dw1
Fr = Ft*np.tan(alp_tw)
Fa = Ft*np.tan(bet)
disp(zv1,zv2,sigF1,sigF2)

array([12.76,  1.72,  1.41,  3.12])

array([ 29.4 , 147.01, 114.11, 117.01])

## Shaft design

In [9]:
# F_r,bet,dw1,dw2,T_sh1,T_sh2 = 1088.95,10.141*np.pi/180,32,128,21575.73,83127.84
# F_r of chain drive (chap 2)
# bet = beta_w (chap 3)
# dw1, dw2 (chap 3)
# t_sh1, t_sh2 (chap 1)
Ft1 = Ft2 = 2*T_sh1/dw1
Fr1 = Fr2 = Ft1*np.tan(alp*np.pi/180)/np.cos(bet)
Fa1 = Fa2 = Ft1*np.tan(bet)
r12, r21 = -dw1/2, dw2/2
hr12, cb12, cq1, hr21, cb21, cq2 = 1,1,1,-1,-1,-1
Fx12 = r12/abs(r12)*cq1*cb12*Ft1
Fx21 = r21/abs(r21)*cq2*cb21*Ft2
Fy12 = -r12/abs(r12)*Ft1*np.tan(alp_t)/np.cos(bet)
Fy21 = -r21/abs(r21)*Ft2*np.tan(alp_t)/np.cos(bet)
Fz12 = cq1*cb12*hr12*Ft1*np.tan(bet)
Fz21 = cq2*cb21*hr21*Ft2*np.tan(bet)
Fy22 = F_r*np.sin(210*np.pi/180)
Fx22 = F_r*np.cos(210*np.pi/180)
d1 = (T_sh1/.2/15)**(1/3)
d2 = (T_sh2/.2/30)**(1/3)
disp(d1,d2)
# d1, d2, bO1, bO2 = 32,28,20,18
d1, d2, bO1, bO2 = 35,40,21,23

lm13,lm12,lm22,lm23,k1,k2,k3,hn= 1.5*d1,1.5*d1,1.5*d2,1.5*d2,10,8,15,18
disp(np.round([lm13,lm12,lm22,lm23],2))
l22 = -(.5*(lm22+bO2)+k3+hn)
l23 = 0.5*(lm23+bO2)+k1+k2
l21 = 2*l23
l12 = -(.5*(lm12+bO1)+k3+hn)
l13 = 0.5*(lm13+bO1)+k1+k2
l11 = 2*l13
C2A2, A2D2, D2B2, A2B2 = -l22, l23, l23, l21
B1C1, A1D1, D1B1, A1B1 = -l12, l13, l13, l11
disp(l12,l13,l11,l22,l23,l21)
# disp(r12,r21,C2A2, A2D2, D2B2, A2B2,B1C1, A1D1, D1B1, A1B1)
disp(Ft1,Fr1,Fa1,Fx12,Fy12,Fz12,Fx21,Fy21,Fz21,Fx22,Fy22)

array([25.55, 34.1 ])

array([[52.5, 52.5, 60. , 60. ]])

array([-69.75,  54.75, 109.5 , -74.5 ,  59.5 , 119.  ])

array([ 2402.28,   899.55,   580.75, -2402.28,   925.46,   580.75,
        2402.28,  -925.46,  -580.75, -2320.05, -1339.48])

In [10]:
from sympy import *
rA2y, rA2x, rB2y, rB2x, rA1y, rA1x, rB1y, rB1x = symbols('r_A2y, r_A2x, r_B2y, r_B2x, r_A1y, r_A1x, r_B1y, r_B1x')
eq1 = rA2x + rB2x + Fx21 + Fx22
eq2 = C2A2*rA2x + (C2A2+A2D2)*Fx21 + (C2A2+A2B2)*rB2x
rA2x,rB2x = np.array(list(linsolve([eq1,eq2],[rA2x,rB2x]))[0],dtype='float')
disp((C2A2+A2D2)*Fx21,C2A2,A2B2)
eq1 = rA2y + rB2y + Fy21 + Fy22
eq2 = C2A2*rA2y + (C2A2+A2D2)*Fy21 + (C2A2+A2B2)*rB2y + dw2*Fz21/2
rA2y,rB2y = np.array(list(linsolve([eq1,eq2],[rA2y,rB2y]))[0],dtype='float')
# disp((C2A2+A2D2)*Fy21+dw2*Fz21/2,C2A2,A2D2,Fy21,dw2,Fz21)
eq1 = rA1x + rB1x + Fx12
eq2 = A1D1*Fx12 + A1B1*rB1x
rA1x,rB1x = np.array(list(linsolve([eq1,eq2],[rA1x,rB1x]))[0],dtype='float')
# disp(A1D1*Fx12)
eq1 = rA1y + rB1y + Fy12
eq2 = A1D1*Fy12 + A1B1*rB1y + dw1*Fz12/2
rA1y,rB1y = np.array(list(linsolve([eq1,eq2],[rA1y,rB1y]))[0],dtype='float')
disp(A1D1*Fy12+dw1*Fz12/2,A1D1,Fy12,dw1,Fz12)
MxA1, MxD1, MxB1, MxC1 = 0, A1D1*rA1x, 0, 0
MyA1, MyD1m, MyD1p, MyB1, MyC1 = 0, A1D1*rA1y, A1D1*rA1y - dw1*Fz12/2, A1B1*rA1y + D1B1*Fy12 - dw1*Fz12/2, 0
MxC2, MxA2, MxD2, MxB2 = 0, C2A2*Fx22, (C2A2+A2D2)*Fx22 + A2D2*rA2x, 0
MyC2, MyA2, MyD2m, MyD2p, MyB2 = 0, C2A2*Fy22, (C2A2+A2D2)*Fy22 + A2D2*rA2y, D2B2*rB2y,0
disp(Fy21+Fy22)
eqM = lambda mx, my, t: np.sqrt(mx**2 + my**2 + 0.75*t**2)
eq = lambda mx, my, t: np.sqrt(mx**2 + my**2)
MeA1 = eqM(MxA1,MyA1,0)
MeD1m = eqM(MxD1,MyD1m,T_sh1)
MeD1p = eqM(MxD1,MyD1p,T_sh1)
MeB1 = eqM(MxB1,MyB1,T_sh1)
MeC1 = eqM(MxC1,MyC1,T_sh1)
MeC2 = eqM(MxC2,MyC2,T_sh2)
MeA2 = eqM(MxA2,MyA2,T_sh2)
MeD2m = eqM(MxD2,MyD2m,T_sh2)
MeD2p = eqM(MxD2,MyD2p,T_sh2)
MeB2 = eqM(MxB2,MyB2,0)
MA1 = eq(MxA1,MyA1,0)
MD1m = eq(MxD1,MyD1m,T_sh1)
MD1p = eq(MxD1,MyD1p,T_sh1)
MB1 = eq(MxB1,MyB1,T_sh1)
MC1 = eq(MxC1,MyC1,T_sh1)
MC2 = eq(MxC2,MyC2,T_sh2)
MA2 = eq(MxA2,MyA2,T_sh2)
MD2m = eq(MxD2,MyD2m,T_sh2)
MD2p = eq(MxD2,MyD2p,T_sh2)
MB2 = eq(MxB2,MyB2,0)

Mes = np.array([MeA1,MeD1m,MeD1p,MeB1,MeC1,MeC2,MeA2,MeD2m,MeD2p,MeB2])
f = interp1d([30,50],[67,55])
ps1,ps2 = f([d1,d2])
sig = np.array([ps1,ps1,ps1,ps1,ps1,ps2,ps2,ps2,ps2,ps2])
# disp(dw1*Fz12/2,dw2*Fz21/2)
disp(rA1x,rA1y,rB1x,rB1y,rA2x,rA2y,rB2x,rB2y)
# disp(MeA1,MeD1m,MeD1p,MeB1,MeC1,MeC2,MeA2,MeD2m,MeD2p,MeB2)
# disp(MA1,MD1m,MD1p,MB1,MC1,MC2,MA2,MD2m,MD2p,MB2)
# np.array((Mes/0.1/sig)**(1/3))
# disp(rA1x,Fx12,rB1x,0,rA1y,Fy12,rB1y,0)
# disp(Fx22,rA2x,Fx21,rB2x,Fy22,rA2y,Fy21,rB2y)
disp(MxA1, MxD1, MxB1, MxC1)
disp(MyA1, MyD1m, MyD1p, MyB1, MyC1)
disp(MxC2, MxA2, MxD2, MxB2)
disp(MyC2, MyA2, MyD2m, MyD2p, MyB2)
disp(T_sh1,T_sh2)

array([321905.88,     74.5 ,    119.  ])

array([62767.9 ,    54.75,   925.46,    41.67,   580.75])

array([-2264.94])

array([ 1201.14,  -352.24,  1201.14,  -573.22,  2571.38,  2132.43,
       -2653.61,   132.51])

array([    0.  , 65762.49,     0.  ,     0.  ])

array([     0.  , -19284.96, -31383.95,      0.  ,      0.  ])

array([      0.  , -172843.64, -157889.73,       0.  ])

array([     0.  , -99791.32, -52610.71,   7884.23,      0.  ])

array([ 50047.56, 237825.99])

## bearing design

In [59]:
X1,Y1,X2,Y2,lh,m1,m2 = .6,1.07,1.66,1,19200,3,3
Q1 = X1*abs(rA1y)+Y1*abs(Fz12)
Q2 = X2*abs(rB2y)+Y1*abs(Fz21)
Cd1 = Q1*L1**(1/m1)
Cd2 = Q2*L2**(1/m2)
FrA1 = np.sqrt(rA1x**2+rA1y**2)
FrB1 = np.sqrt(rB1x**2+rB1y**2)
FrA2 = np.sqrt(rA2x**2+rA2y**2)
FrB2 = np.sqrt(rB2x**2+rB2y**2)
ratioA1 = abs(Fz12)/FrA1
ratioB1 = abs(Fz12)/FrB1
ratioA2 = abs(Fz12)/FrA2
ratioB2 = abs(Fz12)/FrB2
disp(ratioA1, ratioB1, ratioA2,ratioB2)
disp(FrA1,FrB1,FrA2,FrB2)/1e3
d2,D2,r2,C2,Co2=40,90,2.5,31.9,21.7
d1,D1,r1,C1,Co1=35,80,2.5,33.4,25.2
ratio1,ratio2=Fz12/Co1/1e3,Fz12/Co2/1e3
f1 = interp1d([.014,.028,.057,.085,.11,.17,.29,.43,.57],
                 [.3,.34,.37,.41,.45,.48,.52,.54,.54])
f2 = interp1d([.014,.028,.056,.084,.11,.17,.28,.42,.56],
               [.19,.22,.26,.28,.3,.34,.38,.42,.44])
e1, e2 = f1(ratio1),f2(ratio2)
# disp(e1,e2)

f1 = interp1d([.014,.028,.057,.085,.11,.17,.29,.43,.57],
                 [2.3,1.99,1.71,1.55,1.45,1.31,1.15,1.04,1])
f2 = interp1d([.014,.028,.056,.084,.11,.17,.28,.42,.56],
               [1.81,1.62,1.46,1.34,1.22,1.13,1.04,1.01,1])
X1,X2,Y1,Y2 = 0.56,0.45,f1(ratio1),f2(ratio2)
disp(X1,X2,Y1,Y2 )
FsA1 = e1*FrA1
FsB1 = e1*FrB1
FaA1 = FsB1 - Fz12
FaB1 = FsA1 + Fz12
FatA1, FatB1, FatA2, FatB2 = FsA1, FaB1, Fz12, Fz12
QA1 = (X1*FrA1 + Y1*FatA1)
QB1 = (X1*FrB1 + Y1*FatB1)
QA2 = (X2*FrA2 + Y2*FatA2)
QB2 = (X2*FrB2 + Y2*FatB2)
if QA1<QB1: Q1 = QB1
else: Q1 = QA1
if QA2<QB2: Q1 = QB2
else: Q2 = QA2
Qe1 = Q1*(T1**3*t1/(t1+t2)+T2**3*t2/(t1+t2))**(1/3)
Qe2 = Q2*(T1**3*t1/(t1+t2)+T2**3*t2/(t1+t2))**(1/3)
L1 = lh*60*n_sh1*1e-6/2
L2 = lh*60*n_sh2*1e-6/2
Cd1 = Qe1*L1**(1/3)
Cd2 = Qe2*L2**(1/3)
f = interp2d([320,400,500,630,800,1000,1250,1600,2000,2500,3200,4000],
            [8000,12500,16000,20000,25000,32000,40000,50000],
            np.array([[5.75,6.2,6.7,7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4],
                    [6.2,6.7,7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5],
                      [6.7,7.23,7.81,8.41,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6],
                     [7.23,7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8],
                     [7.81,8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2],
                     [8.43,9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6],
                     [9.11,9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6,21.2],
                     [9.83,10.6,11.5,12.4,13.4,14.5,15.6,16.8,18.2,19.6,21.2,22.9]]))
ra1,ra2 = f(n_sh1,lh/2), f(n_sh2,lh/2)
C1 = Qe1*ra1
C2 = Qe2*ra2
XO,YO = .5,.47
Qt11 = XO*Fz12 + YO*FatB1
Qt12 = Fz12
if Qt11<Qt12: Qt1 = Qt12
else: Qt1 = Qt11
XO,YO = .6,.5
Qt21 = XO*Fz12 + YO*FatB2
Qt22 = Fz12
if Qt21<Qt22: Qt2 = Qt22
else: Qt2 = Qt21
disp(ra1,ra2)
disp(Qe1,Qe2,C1,C2,Cd1,Cd2,Qt1,Qt2)/1e3

array([0.46, 0.44, 0.17, 0.22])

array([1.25, 1.33, 3.34, 2.66])

array([0.56, 0.45, 2.1 , 1.64])

array([[12.39],
       [ 7.25]])

array([ 2.17,  1.88, 26.84, 13.66, 25.79, 13.12,  0.76,  0.64])